# STRUCTURAL_FEA

## Overview
The [PyniteFEA](https://github.com/JWock82/Pynite) package used in this function provides a Python interface for 3D static analysis of elastic structures. The current implementation only provides a small subset of the capabilities of the PyniteFEA library, but you can use it as a starting point to implement more advanced structural features such as P-Δ (P-Delta) analysis, tension/compression-only elements, spring supports, plate elements, etc. See the [PyniteFEA documentation](https://pynite.readthedocs.io/en/latest/index.html) for more details.

## Usage
To use this function in Excel, provide lists of nodes, members, materials, sections, supports, and loads as 2D arrays, and specify the output type as a string ('max_disp', 'reactions', or 'chart').

```excel
=STRUCTURAL_FEA(nodes, members, materials, sections, supports, nodal_loads, output)
```

## Arguments
| Argument     | Type         | Required | Description                                      | Example |
|:------------|:-------------|:---------|:-------------------------------------------------|:--------|
| nodes       | list[list]   | Required | List of nodes: [name, x, y, z]                   | [["N1",0,0,0],["N2",168,0,0]] |
| members     | list[list]   | Required | List of members: [name, i-node, j-node, material, section] | [["M1","N1","N2","Steel","W8x24"]] |
| materials   | list[list]   | Required | List of materials: [name, E, G, nu, rho]         | [["Steel",29000,11200,0.3,0.284/12**3]] |
| sections    | list[list]   | Required | List of sections: [name, A, Iy, Iz, J]           | [["W8x24",7.08,18.3,82.7,0.346]] |
| supports    | list[list]   | Required | List of supports: [node, DX, DY, DZ, RX, RY, RZ] (bools) | [["N1",TRUE,TRUE,TRUE,TRUE,FALSE,FALSE]] |
| nodal_loads | list[list]   | Required | List of nodal loads: [node, direction, value]    | [["N2","FY",-5]] |
| output      | string       | Required | Output type: 'max_disp', 'reactions', or 'chart' | "max_disp" |

## Returns
| Output Type   | Type         | Description                                 | Example |
|:-------------|:-------------|:--------------------------------------------|:--------|
| max_disp     | float        | Maximum nodal displacement (model units)     | 0.123   |
| reactions    | list[list]   | Support reactions: [node, DX, DY, DZ, RX, RY, RZ] | [["N1",0.0,5.0,0.0,0.0,0.0,0.0]] |
| chart        | string       | Data URL for PNG image of the deflection diagram | "data:image/png;base64,..." |
| error        | varies       | Error message if calculation fails           | "Error: Invalid input" |

## Examples

### Space Frame Nodal Loads 1
See [PyniteFEA example](https://github.com/JWock82/Pynite/blob/main/Examples/Space%20Frame%20-%20Nodal%20Loads%201.py) for more details.

**nodes: A20:D23**
| name | x    | y    | z    |
|------|------|------|------|
| N1   | 0    | 0    | 0    |
| N2   | -100 | 0    | 0    |
| N3   | 0    | 0    | -100 |
| N4   | 0    | -100 | 0    |

**members: E20:I22**
| name | i-node | j-node | material | section    |
|------|--------|--------|----------|------------|
| M1   | N2     | N1     | Steel    | MySection  |
| M2   | N3     | N1     | Steel    | MySection  |
| M3   | N4     | N1     | Steel    | MySection  |

**materials: K20:O20**
| name  | E     | G     | nu  | rho      |
|-------|-------|-------|-----|----------|
| Steel | 30000 | 10000 | 0.3 | 0.0002836|

**sections: Q20:U20**
| name      | A   | Iy   | Iz   | J   |
|-----------|-----|------|------|-----|
| MySection | 10  | 100  | 100  | 50  |

**supports: W20:AC22**
| node | DX   | DY   | DZ   | RX   | RY   | RZ   |
|------|------|------|------|------|------|------|
| N2   | TRUE | TRUE | TRUE | TRUE | TRUE | TRUE |
| N3   | TRUE | TRUE | TRUE | TRUE | TRUE | TRUE |
| N4   | TRUE | TRUE | TRUE | TRUE | TRUE | TRUE |

**nodal_loads: AE20:AG21**
| node | direction | value  |
|------|-----------|--------|
| N1   | FY        | -50    |
| N1   | MX        | -1000  |

**Output: max_disp**
| max_disp |
|----------|
| 0.0164   |
```excel
=STRUCTURAL_FEA(A20:D23, E20:I22, K20:O20, Q20:U20, W20:AC22, AE20:AG21, "max_disp")
```

**Output: reactions**
| node | DX     | DY    | DZ    | RX    | RY     | RZ    |
|------|--------|-------|-------|-------|--------|-------|
| N2   | -0.213 | 0.318 | 0.0526| 19.98 | -3.17  | 19.0  |
| N3   | 0.0295 | 7.70  | 7.06  | -265  | 0.940  | 0.517 |
| N4   | 0.183  | 42.0  | -7.11 | -236  | -0.0890| -6.07 |
```excel
=STRUCTURAL_FEA(A20:D23, E20:I22, K20:O20, Q20:U20, W20:AC22, AE20:AG21, "reactions")
```

**Output: chart**
| chart |
|-------|
| data:image/png;base64,... |
```excel
=STRUCTURAL_FEA(A20:D23, E20:I22, K20:O20, Q20:U20, W20:AC22, AE20:AG21, "chart")
```

### Space Frame Nodal Loads 2
See [PyniteFEA example](https://github.com/JWock82/Pynite/blob/main/Examples/Space%20Frame%20-%20Nodal%20Loads%202.py) for more details. This example is adapted from *A First Course in the Finite Element Method* by Daryl L. Logan, Problem 5.58. Units are kips and inches.

**nodes: A1:D4**
| name | x    | y    | z    |
|------|------|------|------|
| N1   | 0    | 0    | 0    |
| N2   | 120  | 0    | 0    |
| N3   | 120  | 0    | -120 |
| N4   | 120  | -240 | -120 |

**members: E1:I3**
| name | i-node | j-node | material | section    |
|------|--------|--------|----------|------------|
| M12  | N1     | N2     | Steel    | MySection  |
| M23  | N2     | N3     | Steel    | MySection  |
| M34  | N3     | N4     | Steel    | MySection  |

**materials: K1:O2**
| name  | E     | G     | nu  | rho      |
|-------|-------|-------|-----|----------|
| Steel | 30000 | 10000 | 0.3 | 0.0002836|

**sections: Q1:U2**
| name      | A   | Iy   | Iz    | J   |
|-----------|-----|------|-------|-----|
| MySection | 100 | 200  | 1000  | 100 |

**supports: W1:AC2**
| node | DX   | DY   | DZ   | RX   | RY   | RZ   |
|------|------|------|------|------|------|------|
| N1   | TRUE | TRUE | TRUE | TRUE | TRUE | TRUE |
| N4   | TRUE | TRUE | TRUE | TRUE | TRUE | TRUE |

**nodal_loads: AE1:AG3**
| node | direction | value  |
|------|-----------|--------|
| N2   | FY        | -5     |
| N2   | MX        | -1200  |
| N3   | FZ        | 40     |

**Output: max_disp**
| max_disp |
|----------|
| 3.07     |
```excel
=STRUCTURAL_FEA(A1:D4, E1:I3, K1:O2, Q1:U2, W1:AC2, AE1:AG3, "max_disp")
```

**Output: reactions**
| node | DX    | DY   | DZ    | RX    | RY    | RZ   |
|------|-------|------|-------|-------|-------|------|
| N1   | 8.50  | 2.53 | -32.1 | 13.8  | 2800  | 360  |
| N4   | -8.50 | 2.47 | -7.93 | -1010 | 28.1  | 1980 |
```excel
=STRUCTURAL_FEA(A1:D4, E1:I3, K1:O2, Q1:U2, W1:AC2, AE1:AG3, "reactions")
```

**Output: chart**
| chart |
|-------|
| data:image/png;base64,... |
```excel
=STRUCTURAL_FEA(A1:D4, E1:I3, K1:O2, Q1:U2, W1:AC2, AE1:AG3, "chart")
```

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import io
import base64
import micropip
await micropip.install('PyniteFEA')
from Pynite import FEModel3D

def structural_fea(nodes, members, materials, sections, supports, nodal_loads, output):
    """
    Performs 3D finite element analysis using the Pynite package and returns a single output based on the 'output' argument.

    Args:
        nodes (list[list]): List of nodes: [name, x, y, z].
        members (list[list]): List of members: [name, i-node, j-node, material, section].
        materials (list[list]): List of materials: [name, E, G, nu, rho].
        sections (list[list]): List of sections: [name, A, Iy, Iz, J].
        supports (list[list]): List of supports: [node, DX, DY, DZ, RX, RY, RZ] (bools).
        nodal_loads (list[list]): List of nodal loads: [node, direction, value].
        output (str): Which output to return. One of 'max_disp', 'reactions', or 'chart'.

    Returns:
        max_disp (float): If output == 'max_disp', returns the maximum nodal displacement (model units).
        reactions (list[list]): If output == 'reactions', returns support reactions: [node, DX, DY, DZ, RX, RY, RZ].
        chart (str): If output == 'chart', returns a data URL for PNG image of the deflection diagram.
        If an error occurs, returns the error in the same shape as the requested output.
    """
    try:
        model = FEModel3D()
        # Add materials
        for m in materials:
            model.add_material(str(m[0]), float(m[1]), float(m[2]), float(m[3]), float(m[4]))
        # Add sections
        for s in sections:
            model.add_section(str(s[0]), float(s[1]), float(s[2]), float(s[3]), float(s[4]))
        # Add nodes
        for n in nodes:
            model.add_node(str(n[0]), float(n[1]), float(n[2]), float(n[3]))
        # Add members
        for mem in members:
            model.add_member(str(mem[0]), str(mem[1]), str(mem[2]), str(mem[3]), str(mem[4]))
        # Add supports
        for sup in supports:
            model.def_support(str(sup[0]), bool(sup[1]), bool(sup[2]), bool(sup[3]), bool(sup[4]), bool(sup[5]), bool(sup[6]))
        # Add nodal loads
        for ld in nodal_loads:
            model.add_node_load(str(ld[0]), str(ld[1]), float(ld[2]))
        # Analyze
        model.analyze()
        # Find max displacement
        max_disp = 0.0
        for node in model.nodes.values():
            d = (
                abs(node.DX['Combo 1']) +
                abs(node.DY['Combo 1']) +
                abs(node.DZ['Combo 1'])
            )
            if d > max_disp:
                max_disp = d
        # Get reactions
        reactions = []
        for sup in supports:
            node = model.nodes[str(sup[0])]
            reactions.append([
                str(sup[0]),
                float(node.RxnFX['Combo 1']),
                float(node.RxnFY['Combo 1']),
                float(node.RxnFZ['Combo 1']),
                float(node.RxnMX['Combo 1']),
                float(node.RxnMY['Combo 1']),
                float(node.RxnMZ['Combo 1'])
            ])
        # Generate chart for first member
        chart_url = ""
        if members:
            mname = str(members[0][0])
            member = model.members[mname]
            x, d = member.deflection_array('dy', 50, 'Combo 1')
            fig, ax = plt.subplots(figsize=(7,4))
            ax.plot(x, d, label=f"Deflection (dy)")
            ax.axhline(0, color='black', lw=1)
            ax.set_xlabel('Location along member (in)')
            ax.set_ylabel('Deflection (in)')
            ax.set_title(f"Deflection Diagram for Member {mname}")
            ax.grid(True)
            ax.legend()
            buf = io.BytesIO()
            plt.savefig(buf, format='png', bbox_inches='tight')
            plt.close(fig)
            chart_url = f"data:image/png;base64,{base64.b64encode(buf.getvalue()).decode('utf-8')}"
        # Return only the requested output
        if output == 'max_disp':
            return max_disp
        elif output == 'reactions':
            return reactions
        elif output == 'chart':
            return chart_url
        else:
            return f"Error: Invalid output argument: {output}"
    except Exception as e:
        if output == 'max_disp':
            return None
        elif output == 'reactions':
            return None
        elif output == 'chart':
            return f'Error: {str(e)}'
        else:
            return f'Error: {str(e)}'

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_structural_fea_logan_58():
    nodes = [["N1", 0, 0, 0], ["N2", -100, 0, 0], ["N3", 0, 0, -100], ["N4", 0, -100, 0]]
    members = [["M1", "N2", "N1", "Steel", "MySection"], ["M2", "N3", "N1", "Steel", "MySection"], ["M3", "N4", "N1", "Steel", "MySection"]]
    materials = [["Steel", 30000, 10000, 0.3, 0.0002836]]
    sections = [["MySection", 10, 100, 100, 50]]
    supports = [["N2", True, True, True, True, True, True], ["N3", True, True, True, True, True, True], ["N4", True, True, True, True, True, True]]
    nodal_loads = [["N1", "FY", -50], ["N1", "MX", -1000]]
    expected_max_disp = 0.01641800681769235
    expected_reactions = [
        ["N2", -0.2129477265385237, 0.3178076295329955, 0.05262677121001849, 19.98045220349487, -3.1653593081959004, 18.99066859521227],
        ["N3", 0.02948587214323623, 7.696787649904898, 7.055668005976417, -264.95666927427567, 0.9402728594668361, 0.5167145197604159],
        ["N4", 0.18346185439528745, 41.98540472056211, -7.108294777186436, -235.53202563835256, -0.08900345794916253, -6.072805601201877]
    ]
    max_disp = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'max_disp')
    reactions = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'reactions')
    chart = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'chart')
    assert max_disp == expected_max_disp
    assert reactions == expected_reactions
    assert isinstance(chart, str) and chart.startswith("data:image/png;base64,")

def test_structural_fea_logan_558():
    nodes = [["N1", 0, 0, 0], ["N2", 120, 0, 0], ["N3", 120, 0, -120], ["N4", 120, -240, -120]]
    members = [["M12", "N1", "N2", "Steel", "MySection"], ["M23", "N2", "N3", "Steel", "MySection"], ["M34", "N3", "N4", "Steel", "MySection"]]
    materials = [["Steel", 30000, 10000, 0.3, 0.0002836]]
    sections = [["MySection", 100, 200, 1000, 100]]
    supports = [["N1", True, True, True, True, True, True], ["N4", True, True, True, True, True, True]]
    nodal_loads = [["N2", "FY", -5], ["N2", "MX", -1200], ["N3", "FZ", 40]]
    expected_max_disp = 3.073142865941084
    expected_reactions = [
        ["N1", 8.503544546622555, 2.5255971996715556, -32.07004243280417, 13.81227782813117, 2799.867994372069, 360.32362939116564],
        ["N4", -8.503544546622557, 2.4744028003284377, -7.9299575671939895, -1013.9304299941012, 28.11175196972821, 1983.598725758828]
    ]
    max_disp = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'max_disp')
    reactions = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'reactions')
    chart = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'chart')
    assert max_disp == expected_max_disp
    assert reactions == expected_reactions
    assert isinstance(chart, str) and chart.startswith("data:image/png;base64,")

ipytest.run('-s')

In [ ]:
import gradio as gr

def render_html_chart(chart):
    if isinstance(chart, str) and chart.startswith("data:image/png;base64,"):
        return f'<img src="{chart}" alt="Deflection Chart" style="max-width:100%;height:auto;" />'
    return f'<div style="color:red;">{chart}</div>'

def structural_fea_demo(nodes, members, materials, sections, supports, nodal_loads, output):
    result = structural_fea(nodes, members, materials, sections, supports, nodal_loads, output)
    if output == 'chart':
        html_chart = render_html_chart(result)
        return None, None, html_chart, None if not (isinstance(result, str) and result.startswith('Error')) else result
    elif output == 'max_disp':
        return result, None, None, None if not (result is None) else 'Error: Calculation failed'
    elif output == 'reactions':
        return None, result, None, None if not (result is None) else 'Error: Calculation failed'
    else:
        return None, None, None, f'Error: Invalid output argument: {output}'

examples = [
    [
        # Example 1: 3D Frame (Logan Problem 5.58)
        [["N1", 0, 0, 0], ["N2", 120, 0, 0], ["N3", 120, 0, -120], ["N4", 120, -240, -120]],
        [["M12", "N1", "N2", "Steel", "MySection"], ["M23", "N2", "N3", "Steel", "MySection"], ["M34", "N3", "N4", "Steel", "MySection"]],
        [["Steel", 30000, 10000, 0.3, 0.0002836]],
        [["MySection", 100, 200, 1000, 100]],
        [["N1", True, True, True, True, True, True], ["N4", True, True, True, True, True, True]],
        [["N2", "FY", -5], ["N2", "MX", -1200], ["N3", "FZ", 40]],
        'max_disp'
    ],
    [
        # Example 2: 3D Frame (Logan Example 5.8)
        [["N1", 0, 0, 0], ["N2", -100, 0, 0], ["N3", 0, 0, -100], ["N4", 0, -100, 0]],
        [["M1", "N2", "N1", "Steel", "MySection"], ["M2", "N3", "N1", "Steel", "MySection"], ["M3", "N4", "N1", "Steel", "MySection"]],
        [["Steel", 30000, 10000, 0.3, 0.0002836]],
        [["MySection", 10, 100, 100, 50]],
        [["N2", True, True, True, True, True, True], ["N3", True, True, True, True, True, True], ["N4", True, True, True, True, True, True]],
        [["N1", "FY", -50], ["N1", "MX", -1000]],
        'reactions'
    ]
]

demo = gr.Interface(
    fn=structural_fea_demo,
    inputs=[
        gr.Dataframe(
            type="array",
            label="Nodes",
            value=[["N1", 0, 0, 0], ["N2", 120, 0, 0], ["N3", 120, 0, -120], ["N4", 120, -240, -120]],
            headers=["name", "x", "y", "z"]
        ),
        gr.Dataframe(
            type="array",
            label="Members",
            value=[["M12", "N1", "N2", "Steel", "MySection"], ["M23", "N2", "N3", "Steel", "MySection"], ["M34", "N3", "N4", "Steel", "MySection"]],
            headers=["name", "i-node", "j-node", "material", "section"]
        ),
        gr.Dataframe(
            type="array",
            label="Materials",
            value=[["Steel", 30000, 10000, 0.3, 0.0002836]],
            headers=["name", "E", "G", "nu", "rho"]
        ),
        gr.Dataframe(
            type="array",
            label="Sections",
            value=[["MySection", 100, 200, 1000, 100]],
            headers=["name", "A", "Iy", "Iz", "J"]
        ),
        gr.Dataframe(
            type="array",
            label="Supports",
            value=[["N1", True, True, True, True, True, True], ["N4", True, True, True, True, True, True]],
            headers=["node", "DX", "DY", "DZ", "RX", "RY", "RZ"]
        ),
        gr.Dataframe(
            type="array",
            label="Nodal Loads",
            value=[["N2", "FY", -5], ["N2", "MX", -1200], ["N3", "FZ", 40]],
            headers=["node", "direction", "value"]
        ),
        gr.Radio(["max_disp", "reactions", "chart"], label="Output", value="max_disp")
    ],
    outputs=[
        gr.Number(label="Max Displacement"),
        gr.Dataframe(
            type="array",
            label="Reactions",
            headers=["node", "DX", "DY", "DZ", "RX", "RY", "RZ"]
        ),
        gr.HTML(label="Chart"),
        gr.Textbox(label="Error")
    ],
    description="Finite Element Analysis (STRUCTURAL_FEA) for 3D frames and beams using Pynite. Define nodes, members, materials, sections, supports, and nodal loads. Select output type. Returns max displacement, support reactions, or a deflection plot.",
    flagging_mode='never',
    examples=examples,
)
demo.launch()

In [ ]:
# A First Course in the Finite Element Method, 4th Edition
# Daryl L. Logan
# Problem 5.58
# Units for this model are kips and inches

from Pynite import FEModel3D

# Create a new model
frame = FEModel3D()

# Define the nodes
frame.add_node('N1', 0, 0, 0)
frame.add_node('N2', 10*12, 0, 0)
frame.add_node('N3', 10*12, 0, -10*12)
frame.add_node('N4', 10*12, -20*12, -10*12)

# Define the supports
frame.def_support('N1', True, True, True, True, True, True)
frame.def_support('N4', True, True, True, True, True, True)

# Create members (all members will have the same properties in this example)
J = 100
Iy = 200
Iz = 1000
A = 100
frame.add_section('MySection', A, Iy, Iz, J)

# Define a material
E = 30000
G = 10000
nu = 0.3
rho = 2.836e-4
frame.add_material('Steel', E, G, nu, rho)

frame.add_member('M12', 'N1', 'N2', 'Steel', 'MySection')
frame.add_member('M23', 'N2', 'N3', 'Steel', 'MySection')
frame.add_member('M34', 'N3', 'N4', 'Steel', 'MySection')

# Add nodal loads
frame.add_node_load('N2', 'FY', -5)
frame.add_node_load('N2', 'MX', -100*12)
frame.add_node_load('N3', 'FZ', 40)

# Analyze the frame
frame.analyze()

# Example usage of the refactored function:
nodes = [["N1", 0, 0, 0], ["N2", 120, 0, 0], ["N3", 120, 0, -120], ["N4", 120, -240, -120]]
members = [["M12", "N1", "N2", "Steel", "MySection"], ["M23", "N2", "N3", "Steel", "MySection"], ["M34", "N3", "N4", "Steel", "MySection"]]
materials = [["Steel", 30000, 10000, 0.3, 0.0002836]]
sections = [["MySection", 100, 200, 1000, 100]]
supports = [["N1", True, True, True, True, True, True], ["N4", True, True, True, True, True, True]]
nodal_loads = [["N2", "FY", -5], ["N2", "MX", -1200], ["N3", "FZ", 40]]

max_disp = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'max_disp')
reactions = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'reactions')
chart = structural_fea(nodes, members, materials, sections, supports, nodal_loads, 'chart')

print('Max displacement:', max_disp)
print('Reactions:', reactions)
print('Chart (data URL):', chart[:40] + '...')